# Lab 3 - Nim  
## Task 3.2 - An agent using evolved rules

In [ ]:
import logging
import random
from copy import deepcopy
from nim import Nimply, Nim
from play_nim import nim_sum, random_action, evaluate

In [ ]:
logging.basicConfig(format="%(message)s", level=logging.INFO)

### Implementation

In [ ]:
class EvolvedPlayer():
  """
    This played uses GA to evolve some parameters
    that lets him play the game (hopefully better every time)

    Each individual has a list of probabilities of taking n obj 
    from each row. The number of objects n is decided from another 
    list of probabilities that decides ??????????????????????????????? <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
  """
  def __init__(self, nim: Nim, *, population_size=10, offspring_size=10, n_matches=50):
    assert nim
    assert population_size > 0
    assert offspring_size > 0
    assert n_matches > 1
    
    self.k = nim._k
    self.board = deepcopy(nim._rows)
    self.p_size = population_size
    self.o_size = offspring_size
    self.n_matches = 50

  def initial_population(self):
    self.population = []

    for i in range(self.p_size):
      pass

  def cross_over():
    pass

  def mutation():
    pass

  def genetic_algorithm():
    pass
    
  def ply():
    pass
  